In [13]:
import re


def yaml_parse_structure(lines: [str], self_indent: int = 0) \
        -> (object, [str]):
    group = []
    while True:
        if len(lines) is 0:
            return group, []
        line_indent = re.match(r' *-? ?', lines[0])
        if line_indent.end() < self_indent:
            return group, lines
        elif line_indent.end() > self_indent:
            to_append, next_lines = yaml_parse_structure(
                lines, line_indent.end())
            group.append(to_append)
            lines = next_lines
        else:
            catch_minus = re.search(r'[^ ]+.*$', lines[0])
            group.append(catch_minus[0])
            lines = lines[1:]


In [14]:
def yaml_parse_ignore_minus(structure: list) -> object:
    res = {}

    def remove_minus(string: str) -> str:
        return string[re.match(r'-? ?', string).end():]

    while True:
        if len(structure) is 0:
            return res

        line = remove_minus(structure[0])

        if line[-1] is ':':
            res[line[:-1]] = yaml_parse_meaning(structure[1])
            structure = structure[2:]
        else:
            [key, val] = line.split(':')
            val = re.search(r'[^ ]+.*', val)[0]
            match_number = re.match(r'^-?(\d+\.?\d*|\d*\.?\d+)$', val)
            if match_number is not None:
                match_int = re.match(r'^-?\d+$', val)
                if match_int is not None:
                    val = int(val)
                else:
                    val = float(val)
            res[key] = val
            structure = structure[1:]

In [15]:
def yaml_parse_list(structure: list) -> list:
    delimiters = []
    for pos in enumerate(structure):
        if pos[1][0] is '-':
            delimiters.append(pos[0])
    intervals = [[x] for x in delimiters]
    for i in range(1, len(delimiters)):
        intervals[i - 1].append(delimiters[i])
    intervals[-1].append(-1)
    return [yaml_parse_ignore_minus(structure[a:b])
            for [a, b] in intervals]

In [16]:
def yaml_parse_meaning(structure: list) -> object:
    if len(structure) is 0:
        return None

    if structure[0][0] is '-':
        return yaml_parse_list(structure)
    else:
        return yaml_parse_ignore_minus(structure)


def yaml_parse(lines: [str], self_indent: int = 0) -> object:
    return yaml_parse_meaning(
        yaml_parse_structure(lines, self_indent)[0])

In [17]:
def json_stringify(obj: object) -> str:
    if isinstance(obj, int):
        return '{}'.format(obj)
    elif isinstance(obj, float):
        return '{}'.format(obj)
    elif isinstance(obj, str):
        return '"{}"'.format(obj)
    elif isinstance(obj, list):
        return '[{}]'.format(', '.join([
            json_stringify(elem) for elem in obj]))
    elif isinstance(obj, dict):
        return '{{{}}}'.format(', '.join(['{}: {}'.format(
            json_stringify(key), json_stringify(val))
            for key, val in obj.items()]))


In [24]:
with open('input.yaml', 'r', encoding='utf-8') as file:
    input_lines = file.readlines()

print (input_lines)    

['Monday:\n', '- time:\n', '    begin:\n', '      hour: 8\n', '      minute: 20\n', '    end:\n', '      hour: 9\n', '      minute: 50\n', '  location:\n', '    room: 3207 ауд.\n', '    pavilion: ул. Ломоносова, д. 9, лит. Е\n', '  lesson: Иностранный язык (Прак)\n', '  week: every\n', '\n', '- time:\n', '    begin:\n', '      hour: 11\n', '      minute: 40\n', '    end:\n', '      hour: 13\n', '      minute: 10\n', '  location:\n', '    room: 2438 ауд.\n', '    pavilion: ул. Ломоносова, д. 9, лит. Е\n', '  lesson: Физика (Прак)\n', '  teacher: Чистяков Виктор Владимирович\n', '  week: even\n', '\n', '- time:\n', '    begin:\n', '      hour: 11\n', '      minute: 40\n', '    end:\n', '      hour: 13\n', '      minute: 10\n', '  location:\n', '    room: 2407 ауд.\n', '    pavilion: ул. Ломоносова, д. 9, лит. Е\n', '  lesson: Физика (Лаб)\n', '  teacher: Чистяков Виктор Владимирович\n', '  week: even\n']


In [26]:
input_lines = filter(lambda x: re.match('^\s*$', x) is None,
                     input_lines)

print (input_lines) 

In [27]:
input_lines = [x[:re.search(r'\s*$', x).start()] for x in input_lines]

print (input_lines) 

['Monday:', '- time:', '    begin:', '      hour: 8', '      minute: 20', '    end:', '      hour: 9', '      minute: 50', '  location:', '    room: 3207 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Иностранный язык (Прак)', '  week: every', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2438 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Прак)', '  teacher: Чистяков Виктор Владимирович', '  week: even', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2407 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Лаб)', '  teacher: Чистяков Виктор Владимирович', '  week: even']


In [28]:
with open('output.json', 'w', encoding='utf-8') as file:
    file.write(json_stringify(yaml_parse(input_lines)))
    
print (input_lines)  

['Monday:', '- time:', '    begin:', '      hour: 8', '      minute: 20', '    end:', '      hour: 9', '      minute: 50', '  location:', '    room: 3207 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Иностранный язык (Прак)', '  week: every', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2438 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Прак)', '  teacher: Чистяков Виктор Владимирович', '  week: even', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2407 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Лаб)', '  teacher: Чистяков Виктор Владимирович', '  week: even']


In [30]:
with open('output.json', 'w', encoding='utf-8') as file:
    file.write(json_stringify(yaml_parse(input_lines)))


['Monday:', '- time:', '    begin:', '      hour: 8', '      minute: 20', '    end:', '      hour: 9', '      minute: 50', '  location:', '    room: 3207 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Иностранный язык (Прак)', '  week: every', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2438 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Прак)', '  teacher: Чистяков Виктор Владимирович', '  week: even', '- time:', '    begin:', '      hour: 11', '      minute: 40', '    end:', '      hour: 13', '      minute: 10', '  location:', '    room: 2407 ауд.', '    pavilion: ул. Ломоносова, д. 9, лит. Е', '  lesson: Физика (Лаб)', '  teacher: Чистяков Виктор Владимирович', '  week: even']
